In [11]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

import tempfile, sys, os
sys.path.insert(0, os.path.abspath('..'))

In [12]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [13]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [14]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [15]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

base_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False)

In [16]:
from sam import SAM

In [17]:
optimizer = SAM(base_optimizer)

In [18]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [19]:
@tf.function
def train_step_SAM(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.first_step(gradients, model.trainable_variables)

  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.second_step(gradients, model.trainable_variables)

  train_loss(loss)
  train_accuracy(labels, predictions)

In [20]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [21]:
EPOCHS = 6

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step_SAM(images, labels)
    
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.1826685070991516, Accuracy: 94.55000305175781, Test Loss: 0.06474466621875763, Test Accuracy: 97.97999572753906
Epoch 2, Loss: 0.07989845424890518, Accuracy: 97.61000061035156, Test Loss: 0.05039527639746666, Test Accuracy: 98.32999420166016
Epoch 3, Loss: 0.058620572090148926, Accuracy: 98.21166229248047, Test Loss: 0.05196046084165573, Test Accuracy: 98.50999450683594
Epoch 4, Loss: 0.04371574521064758, Accuracy: 98.72833251953125, Test Loss: 0.05818469822406769, Test Accuracy: 98.40999603271484
Epoch 5, Loss: 0.03423779457807541, Accuracy: 99.01000213623047, Test Loss: 0.0582977756857872, Test Accuracy: 98.5199966430664
Epoch 6, Loss: 0.025284944102168083, Accuracy: 99.26666259765625, Test Loss: 0.06547839939594269, Test Accuracy: 98.5
